# Plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_3d_grid():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_zlim(0, 1)

    ax.grid(False)
    ax.set_xticks(np.linspace(0, 1, 3))
    ax.set_yticks(np.linspace(0, 1, 3))
    ax.set_zticks(np.linspace(0, 1, 3))

    # Plot the grid
    for i in np.linspace(0, 1, 3):
        for j in np.linspace(0, 1, 3):
            ax.plot([i, i], [0, 1], [0, 0], color="black")
            ax.plot([i, i], [0, 1], [1, 1], color="black")
            ax.plot([0, 0], [i, i], [0, 1], color="black")
            ax.plot([1, 1], [i, i], [0, 1], color="black")

    # Plot the observable voxels
    for i in np.linspace(0, 1, 3):
        for j in np.linspace(0, 1, 3):
            for k in np.linspace(0, 1, 3):
                ax.plot([i, i], [j, j], [0, 1], color="black")
                ax.plot([i, i], [0, 1], [j, j], color="black")
                ax.plot([0, 1], [i, i], [j, j], color="black")

    plt.show()


plot_3d_grid()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Define the flow function (sine wave)
def flow(t):
    x = t
    y = np.sin(t)
    z = np.cos(t)
    return np.array([x, y, z])


# Define the normal to the flow (example: derivative of the flow)
def normal(t):
    dx = 1
    dy = np.cos(t)
    dz = -np.sin(t)
    return np.array([dx, dy, dz])


# Define a point in space
point = np.array([1, 1, 1])

# Calculate the vector sum of the flow and the point
t = np.linspace(0, 5 * np.pi, 100)
flow_vectors = flow(t)
normal_vectors = normal(t)
vector_sum = flow_vectors + point[:, np.newaxis]

# Create a figure and a 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# Plot the flow
# ax.plot(flow_vectors[0], flow_vectors[1], flow_vectors[2], label="Flow (Sine Wave)")

# Plot the normal vectors
ax.quiver(
    flow_vectors[0],
    flow_vectors[1],
    flow_vectors[2],
    normal_vectors[0],
    normal_vectors[1],
    normal_vectors[2],
    length=0.1,
    color="r",
    label="Normal Vectors",
)

# Plot the point
ax.scatter(point[0], point[1], point[2], color="g", label="Point")

# Plot the vector sum
ax.plot(vector_sum[0], vector_sum[1], vector_sum[2], label="Vector Sum", linestyle="--")

# Set labels and legend
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend()

plt.show()

## SDFs

In [ ]:
import trimesh
import numpy as np
import torch
import matplotlib.pyplot as plt

directory = "data/SMPL/Colored_1/1.obj" # "data/Exploding_Sphere/10.obj"
# Load the OBJ file
mesh = trimesh.load(directory)

mesh.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=mesh.vertices[:, 0],
            y=mesh.vertices[:, 1],
            z=mesh.vertices[:, 2],
            mode="markers",
            marker=dict(size=0.5, color="black", opacity=1),
            name="Vertices",
        )
    ]
)

fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)
    ),
    margin=dict(r=0, l=0, b=0, t=0),
)

fig.show()

In [ ]:
# Get the bounding box of the mesh
bounding_box = mesh.bounds
min_bound = bounding_box[0]
max_bound = bounding_box[1]

# Create a grid of points with higher resolution
resolution = 256
x = np.linspace(min_bound[0], max_bound[0], resolution)
y = np.linspace(min_bound[1], max_bound[1], resolution)
z = np.linspace(min_bound[2], max_bound[2], resolution)
grid = np.array(np.meshgrid(x, y, z)).T.reshape(-1, 3)


# Convert grid to PyTorch tensor and move to GPU
grid_tensor = torch.tensor(grid, dtype=torch.float32).cuda()

# Convert mesh vertices to PyTorch tensor and move to GPU
vertices_tensor = torch.tensor(mesh.vertices, dtype=torch.float32).cuda()


# Function to compute SDF using PyTorch
def compute_sdf(mesh_vertices, points):
    distances = torch.cdist(points, mesh_vertices)
    sdf = torch.min(distances, dim=1)[0]
    return sdf


# Define batch size
batch_size = 10_000

# Compute the SDF in batches
sdf_list = []
for i in range(0, grid_tensor.shape[0], batch_size):
    batch_points = grid_tensor[i : i + batch_size]
    sdf_batch = compute_sdf(vertices_tensor, batch_points)
    sdf_list.append(sdf_batch)

# Concatenate all batches
sdf_tensor = torch.cat(sdf_list)

# Reshape the SDF to the original grid shape
sdf = sdf_tensor.cpu().numpy().reshape((resolution, resolution, resolution))

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Mesh3d(
            x=mesh.vertices[:, 0],
            y=mesh.vertices[:, 1],
            z=mesh.vertices[:, 2],
            i=mesh.faces[:, 0],
            j=mesh.faces[:, 1],
            k=mesh.faces[:, 2],
            color="cyan",
            opacity=0.50,
        ),
        go.Scatter3d(
            x=mesh.vertices[:, 0],
            y=mesh.vertices[:, 1],
            z=mesh.vertices[:, 2],
            mode="markers",
            marker=dict(size=0.2, color="red", opacity=0.8),
            name="Vertices",
        ),
    ]
)

# Set the layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)
    ),
    margin=dict(r=0, l=0, b=0, t=0),
)

# Display the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
import trimesh
import numpy as np

# Function to generate ellipsoid data
def generate_ellipsoid(center, scale, rotation_matrix):
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 20)
    x = scale[0] * np.outer(np.cos(u), np.sin(v))
    y = scale[1] * np.outer(np.sin(u), np.sin(v))
    z = scale[2] * np.outer(np.ones_like(u), np.cos(v))
    ellipsoid = np.stack([x, y, z], axis=-1)
    ellipsoid = ellipsoid.reshape(-1, 3)
    ellipsoid = ellipsoid @ rotation_matrix.T
    ellipsoid += center
    return ellipsoid


# Sample 10 random points from the mesh vertices
sampled_indices = np.random.choice(mesh.vertices.shape[0], 200, replace=False)
sampled_vertices = mesh.vertices[sampled_indices]

# Create the Plotly figure
fig = go.Figure()

# Add Scatter3d trace for the point cloud of vertices
fig.add_trace(
    go.Scatter3d(
        x=mesh.vertices[:, 0],
        y=mesh.vertices[:, 1],
        z=mesh.vertices[:, 2],
        mode="markers",
        marker=dict(size=0.2, color="black", opacity=0.5),
        name="Vertices",
    )
)

# Add Scatter3d trace for the ellipsoids
for vertex in sampled_vertices:
    scale = np.random.uniform(0.0, 0.025, 3)
    rotation_matrix = trimesh.transformations.random_rotation_matrix()[:3, :3]
    ellipsoid = generate_ellipsoid(vertex, scale, rotation_matrix)
    random_color = np.random.rand(3)
    random_opacity = np.random.uniform(0.1, 0.5)
    fig.add_trace(
        go.Scatter3d(
            x=ellipsoid[:, 0],
            y=ellipsoid[:, 1],
            z=ellipsoid[:, 2],
            mode="markers",
            marker=dict(
                size=2,
                color=f"rgb({random_color[0]*255},{random_color[1]*255},{random_color[2]*255})",
                opacity=random_opacity,
            ),
            name="Ellipsoids",
        )
    )

# Set the layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)
    ),
    margin=dict(r=0, l=0, b=0, t=0),
)

# Display the figure
fig.show()

In [ ]:
# Visualize the SDF slice
slice_index = resolution // 2
plt.imshow(sdf[slice_index, :, :], cmap="viridis")
# plt.title("SDF Slice")
plt.axis("off")
plt.savefig("sdf_slice.png")
plt.show()

In [ ]:
from render import Renderer
import nvdiffrast.torch as dr

renderer = Renderer(50, 1024, dr.RasterizeGLContext(), directory)
target_imgs = renderer.target_imgs

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

name = "target_img_2"
# Save the image as PNG with a transparent background
plt.imsave(f"{name}.png", target_imgs[-25].detach().cpu().numpy().clip(0, 1), cmap='gray', format='png')

# Open the saved image with PIL
img = Image.open(f"{name}.png").convert("RGBA")

# Create a new image with a black background
black_bg = Image.new("RGBA", img.size, (0, 0, 0, 255))

# Paste the original image onto the black background
black_bg.paste(img, (0, 0), img)

# Save the final image as JPG
black_bg.convert("RGB").save(f"{name}.jpg", "JPEG")

## Plotting the color interpolation

In [ ]:
from render import Renderer
import nvdiffrast.torch as dr

R = Renderer(
    1, 1024, dr.RasterizeCudaContext(), "data/SMPL/Colored_1/1.obj", with_texture=True, scale=2
)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(R.target_imgs[0].detach().cpu().numpy().clip(0, 1))
plt.axis("off")
plt.show()

## Plotting flow field (Beta)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from models_animated import EncoderPlusSDF

model = EncoderPlusSDF()
model.load_state_dict(torch.load("logs/1_EXPLODING_SPHERE/best/model.pth"))
model.eval()

beta_model = model.beta_predictor


In [ ]:
from utils import positional_encoding, inverse_sigmoid
import matplotlib.pyplot as plt
import numpy as np
import torch

# Define a grid of points in 3D space
resolution = 10  # Reduced resolution for clarity
x = np.linspace(0, 1, resolution)
y = np.linspace(0, 1, resolution)
z = np.linspace(0, 1, resolution)
t = 0

# Create meshgrid
X, Y, Z = np.meshgrid(x, y, z)

# Reshape for input to the model
xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
xyz_posenc = positional_encoding(torch.from_numpy(xyz).float().cuda(), 64)
time_posenc = positional_encoding(torch.full_like(xyz_posenc[:, :1], t), 64)

# Predict beta for each point
with torch.no_grad():
    beta_predictions = beta_model(torch.cat((xyz_posenc, time_posenc), dim=-1))

beta_predictions = torch.sigmoid(
    inverse_sigmoid(torch.from_numpy(xyz).cuda()) + beta_predictions
)

# Convert predictions back to numpy and reshape
beta_predictions = (
    beta_predictions.cpu().numpy().reshape(resolution, resolution, resolution, 3)
)

# Create a 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection="3d")

# Plot the vector field
ax.quiver(
    X,
    Y,
    Z,
    beta_predictions[:, :, :, 0],
    beta_predictions[:, :, :, 1],
    beta_predictions[:, :, :, 2],
    length=0.1,
    normalize=True,
    color="r",
    alpha=0.7,
)

# Set labels and title
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Beta Predictions Flow Field")

# Show the plot
plt.tight_layout()
plt.savefig("beta_predictions_3d.png", dpi=300, bbox_inches='tight')
plt.close()

# Create 2D slices for better visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (plane, title) in enumerate(
    [("xy", "XY Plane (Z=0.5)"), ("xz", "XZ Plane (Y=0.5)"), ("yz", "YZ Plane (X=0.5)")]
):
    idx = resolution // 2
    if plane == "xy":
        X_2d, Y_2d = np.meshgrid(x, y)
        U = beta_predictions[:, :, idx, 0]
        V = beta_predictions[:, :, idx, 1]
    elif plane == "xz":
        X_2d, Y_2d = np.meshgrid(x, z)
        U = beta_predictions[:, idx, :, 0]
        V = beta_predictions[:, idx, :, 2]
    else:  # yz
        X_2d, Y_2d = np.meshgrid(y, z)
        U = beta_predictions[idx, :, :, 1]
        V = beta_predictions[idx, :, :, 2]

    axes[i].quiver(X_2d, Y_2d, U, V, color="r", alpha=0.7)

    axes[i].set_title(title)
    axes[i].set_xlabel(plane[0].upper())
    axes[i].set_ylabel(plane[1].upper())
    axes[i].set_aspect("equal")

plt.tight_layout()
plt.savefig("beta_predictions_2d.png", dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
np.max(beta_predictions)

In [ ]:
beta_predictions_0 = beta_predictions

In [ ]:
from utils import positional_encoding, inverse_sigmoid
import matplotlib.pyplot as plt
import numpy as np
import torch

# Define a grid of points in 3D space
resolution = 10  # Reduced resolution for clarity
x = np.linspace(0, 1, resolution)
y = np.linspace(0, 1, resolution)
z = np.linspace(0, 1, resolution)
t = 0.8

# Create meshgrid
X, Y, Z = np.meshgrid(x, y, z)

# Reshape for input to the model
xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
xyz_posenc = positional_encoding(torch.from_numpy(xyz).float().cuda(), 64)
time_posenc = positional_encoding(torch.full_like(xyz_posenc[:, :1], t), 64)

# Predict beta for each point
with torch.no_grad():
    beta_predictions = beta_model(torch.cat((xyz_posenc, time_posenc), dim=-1))

beta_predictions = torch.sigmoid(
    inverse_sigmoid(torch.from_numpy(xyz).cuda()) + beta_predictions
)

# Convert predictions back to numpy and reshape
beta_predictions = (
    beta_predictions.cpu().numpy().reshape(resolution, resolution, resolution, 3)
)

# Create a 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection="3d")

# Plot the vector field
ax.quiver(
    X,
    Y,
    Z,
    beta_predictions[:, :, :, 0],
    beta_predictions[:, :, :, 1],
    beta_predictions[:, :, :, 2],
    length=0.1,
    normalize=True,
    color="r",
    alpha=0.7,
)

# Set labels and title
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Beta Predictions Flow Field")

# Show the plot
plt.tight_layout()
plt.show()

# Create 2D slices for better visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (plane, title) in enumerate(
    [("xy", "XY Plane (Z=0.5)"), ("xz", "XZ Plane (Y=0.5)"), ("yz", "YZ Plane (X=0.5)")]
):
    idx = resolution // 2
    if plane == "xy":
        X_2d, Y_2d = np.meshgrid(x, y)
        U = beta_predictions[:, :, idx, 0]
        V = beta_predictions[:, :, idx, 1]
    elif plane == "xz":
        X_2d, Y_2d = np.meshgrid(x, z)
        U = beta_predictions[:, idx, :, 0]
        V = beta_predictions[:, idx, :, 2]
    else:  # yz
        X_2d, Y_2d = np.meshgrid(y, z)
        U = beta_predictions[idx, :, :, 1]
        V = beta_predictions[idx, :, :, 2]

    axes[i].quiver(X_2d, Y_2d, U, V, color="r", alpha=0.7)

    axes[i].set_title(title)
    axes[i].set_xlabel(plane[0].upper())
    axes[i].set_ylabel(plane[1].upper())
    axes[i].set_aspect("equal")

plt.tight_layout()
plt.show()

In [ ]:
np.max(beta_predictions - beta_predictions_0)

In [ ]:
np.max(beta_predictions - beta_predictions_0)